In [1]:
import igibson
from igibson.envs.behavior_mp_env import ActionPrimitives, BehaviorMPEnv
from igibson.utils.ig_logging import IGLogReader
from igibson.transition_model.utils.utils import convert_windows_to_linux_path
import argparse
import datetime
import os
import pprint

import bddl
import h5py
import numpy as np
import json
import igibson
from igibson.activity.activity_base import iGBEHAVIORActivityInstance
from igibson.render.mesh_renderer.mesh_renderer_cpu import MeshRendererSettings
from igibson.render.mesh_renderer.mesh_renderer_vr import VrSettings
from igibson.simulator import Simulator
from igibson.utils.git_utils import project_git_info
from igibson.utils.ig_logging import IGLogReader, IGLogWriter
from igibson.utils.utils import parse_str_config
from igibson.examples.behavior.behavior_replay_using_motion_primitives import get_actions_from_segmentation,get_empty_hand
from igibson.transition_model.utils.utils import convert_windows_to_linux_path
import yaml
import time

INFO:root:Importing iGibson (igibson module)
INFO:root:Assets path: D:\GitHub\behavior-vllm-eval\igibson\data/assets
INFO:root:Gibson Dataset path: D:\GitHub\behavior-vllm-eval\igibson\data/g_dataset
INFO:root:iG Dataset path: D:\GitHub\behavior-vllm-eval\igibson\data/ig_dataset
INFO:root:3D-FRONT Dataset path: D:\GitHub\behavior-vllm-eval\igibson\data/threedfront_dataset
INFO:root:CubiCasa5K Dataset path: D:\GitHub\behavior-vllm-eval\igibson\data/cubicasa_dataset
INFO:root:iGibson Key path: D:\GitHub\behavior-vllm-eval\igibson\data/igibson.key
INFO:root:Example path: D:\GitHub\behavior-vllm-eval\igibson\examples
INFO:root:Example config path: D:\GitHub\behavior-vllm-eval\igibson\examples\configs


Indef: FullArgSpec(args=['parsed_state', 'scope', 'object_map'], varargs=None, varkw=None, defaults=(None, None), kwonlyargs=[], kwonlydefaults=None, annotations={})
torch is not available, falling back to rendering to memory(instead of tensor)


In [2]:
demo_path="D:\GitHub\behavior-vllm-eval\igibson\data\virtual_reality\bottling_fruit_0_Benevolence_1_int_0_2021-09-10_14-29-04.hdf5"
segmentation_path="D:\GitHub\behavior-vllm-eval\igibson\data\virtual_reality_segmentation_results\bottling_fruit_0_Benevolence_1_int_0_2021-09-10_14-29-04.json"
demo_path=convert_windows_to_linux_path(demo_path)
segmentation_path=convert_windows_to_linux_path(segmentation_path)


In [3]:
task = IGLogReader.read_metadata_attr(demo_path, "/metadata/atus_activity")
task_id = IGLogReader.read_metadata_attr(demo_path, "/metadata/activity_definition")
scene_id = IGLogReader.read_metadata_attr(demo_path, "/metadata/scene_id")

# Load the segmentation of a demo for this task.
with open(segmentation_path, "r") as f:
    selected_demo_data = json.load(f)

# Get the actions from the segmentation
actions = get_actions_from_segmentation(selected_demo_data)

# Prepare the environment
config_file = os.path.join(igibson.example_config_path, "behavior_segmentation_replay.yaml")
with open(config_file, "r") as f:
    config = yaml.safe_load(f)

config["task"] = task
config["task_id"] = task_id
config["scene_id"] = scene_id

env = BehaviorMPEnv(
    config_file=config,
    mode="simple",
    action_timestep=1.0 / 300.0,
    physics_timestep=1.0 / 300.0,
    use_motion_planning=False,
    activity_relevant_objects_only=False,
)

start = time.time()
env.reset()

# env.robots[0].set_position_orientation([0, 0, 0.7], [0, 0, 0, 1])
done = False
infos = []
action_successes = []
# import pybullet as p

# p.configureDebugVisualizer(p.COV_ENABLE_GUI, False)
# p.resetDebugVisualizerCamera(cameraTargetPosition=[1, -1, 0], cameraDistance=4, cameraYaw=240, cameraPitch=-45)


Conversion of demo segmentation to motion primitives:
Found segment with multiple state changes, using the first: {'start': 1, 'end': 663, 'duration': 662, 'state_records': [{'name': 'InReachOfRobot', 'objects': ['peach_60_multiplexer'], 'value': True}, {'name': 'InReachOfRobot', 'objects': ['fridge_27'], 'value': True}], 'sub_segments': []}
Found segment with multiple state changes, using the first: {'start': 4980, 'end': 5003, 'duration': 23, 'state_records': [{'name': 'OnTop', 'objects': ['strawberry_59_multiplexer', 'jar_61'], 'value': True}, {'name': 'Inside', 'objects': ['strawberry_59_multiplexer', 'jar_61'], 'value': True}], 'sub_segments': []}
ActionPrimitives.RIGHT_GRASP(jar_62)
ActionPrimitives.OPEN(jar_62)
ActionPrimitives.RIGHT_PLACE_ONTOP(countertop_18)
ActionPrimitives.RIGHT_GRASP(jar_61)
ActionPrimitives.RIGHT_PLACE_ONTOP(countertop_18)
ActionPrimitives.OPEN(jar_61)
ActionPrimitives.OPEN(fridge_27)
ActionPrimitives.RIGHT_GRASP(strawberry_59_multiplexer)
ActionPrimitives

INFO:root:IndoorScene model: Benevolence_1_int
INFO:root:StaticIndoorScene scene: Benevolence_1_int
INFO:root:Category walls
INFO:root:Loading the following URDF template D:\GitHub\behavior-vllm-eval\igibson\data/ig_dataset\scenes\Benevolence_1_int\urdf\Benevolence_1_int_walls.urdf
INFO:root:Scale: [1. 1. 1.]
INFO:root:Number of splits: 1
INFO:root:Instantiating scene into the following urdfs:
INFO:root:D:\GitHub\behavior-vllm-eval\igibson\data/ig_dataset\scene_instances\20240412-192257_14109396380895493772_51764\walls_0.urdf
INFO:root:Category floors
INFO:root:Loading the following URDF template D:\GitHub\behavior-vllm-eval\igibson\data/ig_dataset\scenes\Benevolence_1_int\urdf\Benevolence_1_int_floors.urdf
INFO:root:Scale: [1. 1. 1.]
INFO:root:Number of splits: 1
INFO:root:Instantiating scene into the following urdfs:
INFO:root:D:\GitHub\behavior-vllm-eval\igibson\data/ig_dataset\scene_instances\20240412-192257_14109396380895493772_51764\floors_0.urdf
INFO:root:Category ceilings
INFO:

indent from and: 1
indent from and: 1
SCENE NAME: Benevolence_1_int


INFO:root:Instantiating scene into the following urdfs:
INFO:root:D:\GitHub\behavior-vllm-eval\igibson\data/ig_dataset\scene_instances\20240412-192257_14109396380895493772_51764\straight_chair_2_0.urdf
INFO:root:Category straight_chair
INFO:root:Loading the following URDF template D:\GitHub\behavior-vllm-eval\igibson\data/ig_dataset\objects\straight_chair\8ec95f15623085a7b11ae648ea92233\8ec95f15623085a7b11ae648ea92233_simplified.urdf
INFO:root:Scale: [0.92288527 0.82388363 0.95336229]
INFO:root:Number of splits: 1
INFO:root:Instantiating scene into the following urdfs:
INFO:root:D:\GitHub\behavior-vllm-eval\igibson\data/ig_dataset\scene_instances\20240412-192257_14109396380895493772_51764\straight_chair_3_0.urdf
INFO:root:Category straight_chair
INFO:root:Loading the following URDF template D:\GitHub\behavior-vllm-eval\igibson\data/ig_dataset\objects\straight_chair\8ec95f15623085a7b11ae648ea92233\8ec95f15623085a7b11ae648ea92233_simplified.urdf
INFO:root:Scale: [0.92288527 0.82388363 0.

Texture:  D:\GitHub\behavior-vllm-eval\igibson\data/ig_dataset\scenes\Benevolence_1_int\shape/visual\../../material/wall/COMBINED.png
Texture:  D:\GitHub\behavior-vllm-eval\igibson\data/ig_dataset\scenes\Benevolence_1_int\shape/visual\../../material/wall/NORMAL.png
Texture:  D:\GitHub\behavior-vllm-eval\igibson\data/ig_dataset\scenes\Benevolence_1_int\shape/visual\../../material/floor_0/COMBINED.png
Texture:  D:\GitHub\behavior-vllm-eval\igibson\data/ig_dataset\scenes\Benevolence_1_int\shape/visual\../../material/floor_0/NORMAL.png
Texture:  D:\GitHub\behavior-vllm-eval\igibson\data/ig_dataset\scenes\Benevolence_1_int\shape/visual\../../material/ceiling/COMBINED.png
Texture:  D:\GitHub\behavior-vllm-eval\igibson\data/ig_dataset\scenes\Benevolence_1_int\shape/visual\../../material/ceiling/NORMAL.png
Texture:  D:\GitHub\behavior-vllm-eval\igibson\data/ig_dataset\objects\shelf\b7697d284c38fcade76bc197b3a3ffc0\shape/visual\../../material/DIFFUSE.encrypted.png
Texture:  D:\GitHub\behavior-v

In [4]:
print(actions)

[(<ActionPrimitives.RIGHT_GRASP: 2>, 'jar_62'), (<ActionPrimitives.OPEN: 7>, 'jar_62'), (<ActionPrimitives.RIGHT_PLACE_ONTOP: 4>, 'countertop_18'), (<ActionPrimitives.RIGHT_GRASP: 2>, 'jar_61'), (<ActionPrimitives.RIGHT_PLACE_ONTOP: 4>, 'countertop_18'), (<ActionPrimitives.OPEN: 7>, 'jar_61'), (<ActionPrimitives.OPEN: 7>, 'fridge_27'), (<ActionPrimitives.RIGHT_GRASP: 2>, 'strawberry_59_multiplexer'), (<ActionPrimitives.CLOSE: 8>, 'fridge_27'), (<ActionPrimitives.LEFT_GRASP: 1>, 'carving_knife_63'), (<ActionPrimitives.LEFT_PLACE_ONTOP: 3>, 'countertop_18'), (<ActionPrimitives.LEFT_GRASP: 1>, 'carving_knife_63'), (<ActionPrimitives.RIGHT_PLACE_ONTOP: 4>, 'room_floor_kitchen_0'), (<ActionPrimitives.LEFT_PLACE_ONTOP: 3>, 'room_floor_kitchen_0'), (<ActionPrimitives.RIGHT_GRASP: 2>, 'jar_61'), (<ActionPrimitives.CLOSE: 8>, 'jar_61'), (<ActionPrimitives.RIGHT_PLACE_ONTOP: 4>, 'countertop_18'), (<ActionPrimitives.OPEN: 7>, 'fridge_27'), (<ActionPrimitives.RIGHT_GRASP: 2>, 'peach_60_multiplexer

In [4]:
action_pair=actions[0]

In [5]:
primitive, obj_name = action_pair
print(primitive, obj_name)

ActionPrimitives.RIGHT_GRASP jar_62


In [6]:
for i, obj in enumerate(env.addressable_objects):
    print(obj.name)
    if obj.name == obj_name:
        obj_id = i
        break


picture_40
bowl-68_0-0
picture_39
shelf_41
bowl-68_3-1
left_hand_1
mug-08_0-1
shelf_42
right_hand_1
video_game-Mario_Luigi_Dream_Team_Nintendo_3DS_Game-1
BREye_1
shelf_47
strawberry_59_multiplexer
board_game-Hasbro_Life_Board_Game_White_Colorful_Box-10
board_game-Mad_Gab_Refresh_Card_Game_Orange_Purple_Cube-3
countertop_18
countertop_23
oven_24
bottom_cabinet_no_top_17
picture_38
dishtowel-Tag_Dishtowel_Green-2
jar_61
board_game-Balderdash_Game_Blue_Purple_Medium_Size-12
peach_60_multiplexer
fridge_27
window_53
video_game-Super_Mario_3D_World_Wii_U_Game-3
video_game-Mario_Party_9_Wii_Game-10
wall_mounted_tv_49
chopping_board-10_0-0
countertop_21
shelf_7
video_game-Persona_Q_Shadow_of_the_Labyrinth_Nintendo_3DS-9
hand_towel-Threshold_Hand_Towel_Light_Yellow-1
floor_lamp_8
toy-Transformers_Age_of_Extinction_Stomp_and_Chomp_Grimlock_Figure-3
board_game-Tetris_Link_Game_Dark_Purple_Yellow_White-8
door_54
board_game-Pass_The_Popcorn_Movie_Guessing_Game_Yellow_Red_White_Box-7
pop-29-0
door_5

In [12]:
obj=env.scene.objects_by_name['oven_24']
print(obj.name,obj.category)
objs=env.scene.objects_by_category['window']
print(objs)

oven_24 oven
[<igibson.objects.articulated_object.URDFObject object at 0x0000026CAD8F2CD0>, <igibson.objects.articulated_object.URDFObject object at 0x0000026CAFB3E940>, <igibson.objects.articulated_object.URDFObject object at 0x0000026CAFBE3550>]


: 

In [24]:
action = int(primitive) * env.num_objects + obj_id

In [25]:
state, reward, done, info = env.step(action)

PRIMITIVE: grasp jar_61 success, obj in hand <igibson.objects.articulated_object.URDFObject object at 0x00000290AB003FD0>
PRIMITIVE satisfied predicates: {'satisfied': [], 'unsatisfied': [0, 1, 2, 3, 4]}


In [29]:
for goal in env.task.natural_language_goal_conditions:
    print(goal)

for at least one jar,
        strawberry1 is inside the jar, and
        the following is NOT true:
            peach1 is inside the jar
for at least one jar,
        peach1 is inside the jar, and
        the following is NOT true:
            strawberry1 is inside the jar
for every jar,
    the following is NOT true:
        the jar is open
strawberry1 is sliced
peach1 is sliced


In [37]:
env.task.object_scope

{'strawberry.n.01_1': <igibson.objects.multi_object_wrappers.ObjectMultiplexer at 0x290ab0adf10>,
 'electric_refrigerator.n.01_1': <igibson.objects.articulated_object.URDFObject at 0x290aadefb80>,
 'peach.n.03_1': <igibson.objects.multi_object_wrappers.ObjectMultiplexer at 0x290ab1a6670>,
 'countertop.n.01_1': <igibson.objects.articulated_object.URDFObject at 0x290aad296d0>,
 'jar.n.01_1': <igibson.objects.articulated_object.URDFObject at 0x290ab003fd0>,
 'jar.n.01_2': <igibson.objects.articulated_object.URDFObject at 0x290aafa3d30>,
 'carving_knife.n.01_1': <igibson.objects.articulated_object.URDFObject at 0x290aaff9d30>,
 'cabinet.n.01_1': <igibson.objects.articulated_object.URDFObject at 0x290aad11cd0>,
 'floor.n.01_1': <igibson.object_states.on_floor.RoomFloor at 0x290ab1abe20>,
 'agent.n.01_1': <igibson.robots.behavior_robot.BRBody at 0x290b09e67f0>}

In [43]:
env.task.object_scope['strawberry.n.01_1'].states

{igibson.object_states.temperature.Temperature: <igibson.objects.multi_object_wrappers.ObjectGrouper.AbsoluteStateAggregator at 0x290ab1a62e0>,
 igibson.object_states.max_temperature.MaxTemperature: <igibson.objects.multi_object_wrappers.ObjectGrouper.AbsoluteStateAggregator at 0x290ab1a6250>,
 igibson.object_states.adjacency.HorizontalAdjacency: <igibson.objects.multi_object_wrappers.ObjectGrouper.AbsoluteStateAggregator at 0x290ab1a6460>,
 igibson.object_states.room_states.InsideRoomTypes: <igibson.objects.multi_object_wrappers.ObjectGrouper.AbsoluteStateAggregator at 0x290ab1a6220>,
 igibson.object_states.contact_bodies.ContactBodies: <igibson.objects.multi_object_wrappers.ObjectGrouper.AbsoluteStateAggregator at 0x290ab1a6280>,
 igibson.object_states.aabb.AABB: <igibson.objects.multi_object_wrappers.ObjectGrouper.AbsoluteStateAggregator at 0x290ab1a62b0>,
 igibson.object_states.adjacency.VerticalAdjacency: <igibson.objects.multi_object_wrappers.ObjectGrouper.AbsoluteStateAggregator

In [36]:
print(env.task.object_scope['strawberry.n.01_1'].states[igibson.object_states.sliced.Sliced].get_value())
env.task.object_scope['strawberry.n.01_1'].states[igibson.object_states.sliced.Sliced].set_value(True)
print(env.task.object_scope['strawberry.n.01_1'].states[igibson.object_states.sliced.Sliced].get_value())

False
True


In [38]:
print(env.task.object_scope['peach.n.03_1'].states[igibson.object_states.sliced.Sliced].get_value())
env.task.object_scope['peach.n.03_1'].states[igibson.object_states.sliced.Sliced].set_value(True)
print(env.task.object_scope['peach.n.03_1'].states[igibson.object_states.sliced.Sliced].get_value())

False
True


In [49]:
igibson.debug_sampling=True
print(env.task.object_scope['strawberry.n.01_1'].states[igibson.object_states.inside.Inside].get_value(env.task.object_scope['jar.n.01_2']))
x=env.task.object_scope['strawberry.n.01_1'].states[igibson.object_states.inside.Inside].set_value(env.task.object_scope['jar.n.01_2'],True)
print(x)
print(env.task.object_scope['strawberry.n.01_1'].states[igibson.object_states.inside.Inside].get_value(env.task.object_scope['jar.n.01_2']))
print(env.task.object_scope['jar.n.01_2'].states[igibson.object_states.open.Open].get_value())
env.task.object_scope['jar.n.01_2'].states[igibson.object_states.open.Open].set_value(True)
print(env.task.object_scope['jar.n.01_2'].states[igibson.object_states.open.Open].get_value())
print(env.task.object_scope['strawberry.n.01_1'].states[igibson.object_states.inside.Inside].get_value(env.task.object_scope['jar.n.01_2']))
x=env.task.object_scope['strawberry.n.01_1'].states[igibson.object_states.inside.Inside].set_value(env.task.object_scope['jar.n.01_2'],True)
print(x)
print(env.task.object_scope['strawberry.n.01_1'].states[igibson.object_states.inside.Inside].get_value(env.task.object_scope['jar.n.01_2']))

False
None
False
True
True
False
None
False


In [50]:
info=env.task.check_success()
print(info)

(False, {'satisfied': [3, 4], 'unsatisfied': [0, 1, 2]})


In [42]:
env.task.object_scope['jar.n.01_2'].states

{igibson.object_states.adjacency.HorizontalAdjacency: <igibson.object_states.adjacency.HorizontalAdjacency at 0x290ab00c670>,
 igibson.object_states.room_states.InsideRoomTypes: <igibson.object_states.room_states.InsideRoomTypes at 0x290ab00c430>,
 igibson.object_states.contact_bodies.ContactBodies: <igibson.object_states.contact_bodies.ContactBodies at 0x290ab00c3d0>,
 igibson.object_states.aabb.AABB: <igibson.object_states.aabb.AABB at 0x290ab00c190>,
 igibson.object_states.adjacency.VerticalAdjacency: <igibson.object_states.adjacency.VerticalAdjacency at 0x290ab00c400>,
 igibson.object_states.pose.Pose: <igibson.object_states.pose.Pose at 0x290ab00c4c0>,
 igibson.object_states.dirty.Stained: <igibson.object_states.dirty.Stained at 0x290ab00c070>,
 igibson.object_states.open.Open: <igibson.object_states.open.Open at 0x290ab00c1f0>,
 igibson.object_states.dirty.Dusty: <igibson.object_states.dirty.Dusty at 0x290ab00c610>,
 igibson.object_states.inside.Inside: <igibson.object_states.ins

In [41]:
env.task.object_scope['jar.n.01_1'].states

{igibson.object_states.adjacency.HorizontalAdjacency: <igibson.object_states.adjacency.HorizontalAdjacency at 0x290ab00c580>,
 igibson.object_states.room_states.InsideRoomTypes: <igibson.object_states.room_states.InsideRoomTypes at 0x290ab00cb50>,
 igibson.object_states.contact_bodies.ContactBodies: <igibson.object_states.contact_bodies.ContactBodies at 0x290ab00ca90>,
 igibson.object_states.aabb.AABB: <igibson.object_states.aabb.AABB at 0x290ab00c7c0>,
 igibson.object_states.adjacency.VerticalAdjacency: <igibson.object_states.adjacency.VerticalAdjacency at 0x290ab00c160>,
 igibson.object_states.pose.Pose: <igibson.object_states.pose.Pose at 0x290ab00c5e0>,
 igibson.object_states.dirty.Stained: <igibson.object_states.dirty.Stained at 0x290ab00c880>,
 igibson.object_states.open.Open: <igibson.object_states.open.Open at 0x290ab00c910>,
 igibson.object_states.dirty.Dusty: <igibson.object_states.dirty.Dusty at 0x290ab00c970>,
 igibson.object_states.inside.Inside: <igibson.object_states.ins

In [39]:
env.task.simulator.step()

In [33]:
for obj in env.addressable_objects:
    print(obj.name)

hand_towel-Threshold_Hand_Towel_Light_Yellow-0
mug-08_0-0
door_55
bowl-68_1-3
pop-17-5
walls
left_hand_1
jar_61
top_cabinet_31
right_hand_1
BREye_1
microwave_36
picture_39
straight_chair_2
video_game-Super_Mario_3D_World_Wii_U_Game-3
stool_13
sausage-sausage_000-1
bottom_cabinet_no_top_25
straight_chair_5
plate-plate_000-3
picture_43
door_52
picture_40
board_game-Sorry_Sliders_Board_Game_Orage_Purple_Box-9
floors
board_game-Mad_Gab_Refresh_Card_Game_Orange_Purple_Cube-3
board_game-Balderdash_Game_Blue_Purple_Medium_Size-12
video_game-Mario_Party_9_Wii_Game-10
board_game-Mattel_SKIP_Card_Game_Green_Orange_Small_Box-2
board_game-My_Monopoly_Board_Game_Green_Red_Box-5
peach_60_multiplexer
picture_38
board_game-Hasbro_Life_Board_Game_White_Colorful_Box-10
top_cabinet_28
board_game-Hasbro_Trivial_Pursuit_Family_Edition_Game_Light_Blue_Black_Medium_Box-0
countertop_18
mug-08_0-1
bottom_cabinet_no_top_16
chopping_board-10_0-0
dishtowel-Tag_Dishtowel_Basket_Weave_Red-3
BRBody_1
board_game-Pass

In [34]:
for obj in env.addressable_objects:
    if obj.name=="window_50":
        print(obj.states[igibson.object_states.dirty.Dusty].get_value())
        obj.states[igibson.object_states.dirty.Dusty].set_value(True)
        print(obj.states[igibson.object_states.dirty.Dusty].get_value())

env.task.simulator.step()

False
True


D:\GitHub\behavior-vllm-eval\igibson\utils\sampling_utils.py:451: RuntimeWarning: divide by zero encountered in true_divide
  multiple_to_face_on_each_axis = closer_point_on_each_axis / ray_direction


In [35]:
for obj in env.addressable_objects:
    if obj.name=="window_50":
        print(obj.states[igibson.object_states.dirty.Dusty].get_value())

True
